In [5]:
kk 0

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^!kk0:start>^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


# ex1 VASP 基本输入文件的准备

Windows 下面编辑的文件如果想在 linux 服务器上运行, 由于格式问题, 最好先执行: `dos2linux` 这个命令;

**INCAR:**   
* SYSTEM = O atom # 氧 符号，不是零！  
* ISMAER = 0 # 是零，不是氧符号！  
* SIGMA = 0.0

**ISMEAER**,  
不同的值对应的是不同的展宽方法:  
* 对于半导体和绝缘体体系,  ISMEAR 的值取绝对不能大于 0, 一般用 0, 更加精确的时候用 -5, 如果体系非常大, 只有 1 个或者 2 个 K 点的时候, 则用 ISMEAR = 0;
* 对于金属来说,  ISMEAR 的取值一般为 1;  
* 一般来说,  ISMEAR = 0 （Gaussian Smearing） 可以满足大部分的 体系（金属, 导体, 半导体, 分子）;   
* 在 DOS 能带计算中,  使用 ISMEAR = -5 用于获取精确的信息;  

**SIGMA**:
* 如果用了 ISMEAR = -5 ， 可以忽略；
* 对于金属： ISMEAR = 1 或者 0，非金属： ISMEAR = 0 的时候，一般 取 0.10 或者采用默认值 0.20 即可。
* 不过很多人都不放心，为了保险想测试一番。 标准是： SIGMA 的取值要保证 OUTCAR 中的 entropy T\*S 这一项，平均到每个原子上，要 小于 1-2 meV。检查的时候可以用这个命令： grep 'entropy T' OUTCAR得出的能量除以体系中原子的数目再和 0.001 eV 比较，如果小于，SIGMA 取值 OK，如果大于，再换个测试。
* 对于分子，原子体系，必须要用很小的值，这里我们用的 0.01。 

算原子或者分子时， ISMEAR 选择 0，SIGMA 要足够小。

**KPOINTS**:  
K-POINTS # 第一行随便写都行  
0 # 零，格子自动生成  
Gamma # gamma 点 centered  
1 1 1 # 1*1*1 格子  
0 0 0 # S1 S2 S3, 一般保持 0 0 0 不变。

* 另外一种是原始的 Monkhorst-Pack 网格，两者的区别是 M 或者 m 在 G 的基础上在三个方向上平移了 1/(2N)个单位。其中 G，也叫 gamma centered Monkhorst-Pack Grid. 具体见扩展阅读 KPOINTS；
* 对于原子或者分子的计算， K 点取一个 gamma 点就够了，也就是这个 KPOINTS 文件可以用于绝大多数的原子或者分子计算，不用再修改；
* K 点在 VASP 计算中非常重要，大师兄给的一个建议是，永远用 gama centered，也就是第三行保持 G 不变。因为 M 平移之后，网格的对称性和晶胞的对称性会出现不匹配的情况，从而导致计算出错，尤其是对于六角晶系 hexagonal 的结构，必须用 gama centered！

**POSCAR:    **

O atom in a box #1   
1.0 ! universal scaling parameters #2  
8.0 0.0 0.0 ! lattice vector a(1) #3   
0.0 8.0 0.0 ! lattice vector a(2) #4   
0.0 0.0 8.0 ! lattice vector a(3) #5   
O ! O element #6 not zero   
1 ! number of atoms #7   
cart ! positions in cartesian coordinates #8   
0 0 0 #9

* 把氧原子放到一个长宽高均为 8.0 \AA （埃）的格子里，如果你要算其他原子或者分子，均采用这种模型， 格子长宽高可以随便更改。
* 第六行是 氧的元素符号，这里需要注意的是， vasp4.xx 版本里面没有这一行，最新的版本里面有;  在 POSCAR 里面该行没有的话也不会影响计算， VASP 会读取POTCAR 中的元素信息，但输出文件 CONTCAR 中会把该行自动添上；
* C 或者c 代表笛卡尔坐标， D 或者 d 代表分数坐标系，两者的转换见参考阅读POSCAR2， 值得注意的是， VASP 输出文件 CONTCAR 里面采用的是分数坐标系
* 如果想实现两个坐标系之间转换，我们只需改变第 7 行后面的部分即可；
* 查看 POTCAR 中的元素： `grep TIT POTCAR`; 查看 POTCAR 的 Cut-off 能量： `grep ENMAX POTCAR`

In [6]:
electronic_structure('O')

8: 1s2 2s2 2p4


# ex2 做计算常用的批量处理方法

批量生成多个包含相同文件的目录:  
* `for i in {2..9}; do cp -r 0.01 0.0$i; done`  
* `for i in {1..6}; do cp -r ../sigma/0.01 $i$i$i; done*` 

# Ex3 做计算常用的批量处理方法 2

批量替换多个目录内的某个文件中的字符串(将每个文件夹中 INCAR 的 SIGMA 值替换成文件夹的值):  
`for i in * ; do sed -i "3s/0.01/$i/g" $i/INCAR; done`;  
* -i 表示直接对源文件进行编辑;
* 3s 表示的是选择第三行，因为我们知道 0.01 在第三行中出现， s 是 select 的缩写；
* g 代表 global ，意思是全部替换。
*  \$i/INCAR: 要替换的文件
* 单引号和双引号的区别: 单引号中的所有内容都会被当做字符来处理('\$i'是个出字符), 而使用双引号，则可以读取变量 \$i 的值;

# Ex4 做计算常用的批量处理方法 3

sed 多行替换

# Ex5 VASP 运行以及 OSZICAR 详解

这些参数描述的应该是每一个电子步的具体计算过程: 每个电子步的具体计算过程?

* N(iteration  count): 代表电子步的迭代步数
* E: 代表当前电子步的体系能量
* dE: 该步和上一步体系能量的差值
* d eps: the change in the bandstructure energy；本征值的变化
* ncg: the number of evaluations of the Hamiltonian acting onto a wavefunction, 波函数的优化次数: 哈密顿算符作用在一个波函数上的的次数?;
* rms: the norm of the residuum of the trialwavefunctions (their approximate error)
* rms(c): the difference between input andoutput charge density.

电子自洽算法:  
* DAV: blockedDavidson iteration scheme;
* RMM: residual minimization scheme;
* CG: conjugate-gradient algorithm;

tail OSZICAR: **1 F= -.21470938E-01 E0= -.13757822E-01  d E =-.154262E-01**

* num (F): 离子步的步数;  
* F: 体系的总能量, OUTCAR中的`free energy    TOTEN`;
* E0: OUTCAR中`energy(sigma->0)`的能量;
* 计算完成后我们需要获得的体系能量: 查询 OSZICAR 中 E0; 或者 OUTCAR 中 energy(sigma->0);

OUTCAR:  
* 体系总能: `grep 'free energy    TOTEN' OUTCAR | tail -n 1`
* sigma趋于0时的能量: `grep 'energy(sigma->0)' OUTCAR | tail -n 1`
* 不考虑熵变(`T*S`)时的能量: `grep 'energy  without entropy' OUTCAR`
* 熵变: `grep 'entropy T\*S' OUTCAR | tail -n 1`

验证 `energy  without entropy = free energy TOTEN - entropy T*S`:

In [13]:
energy_without_entropy = -0.00604471
free_energy_TOTEN = -0.02147094
entropy_TS = -0.01542623
free_energy_TOTEN - entropy_TS == energy_without_entropy

True

In [17]:
kk 1

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>!kk1:active>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


# Ex6 OUTCAR 的基本内容

OUTCAR的基本结构以及输出内容:  
1. VASP 会列出来其版本，时间，以及服务器的相关情况;  
1. `POTCAR` 的部分信息;
    * **ZVAL** 是该 POTCAR 中对应元素的价电子，这里氧原子含有 6 个外层价电子。
1. `POSCAR` 的一些基本信息： 坐标格式，原子位置，以及晶胞的形状大小。
1. `KPOINTS:` K 点信息：K 点个数..
1. 计算中参数详情: `Dimension of arrays`
1. 本计算的文字描述，任务类型：`Static calculation`
1. 体系大小, `direct lattice vectors`, `reciprocal lattice vectors` ...
1. 计算量的大小: `plane waves:`平面波的个数..
1. 内存占用信息`total amount of memory used by VASP on root node`
1. 电子步迭代的详细过程, `Iteration    1(   1)` 会在OSZCAR中同步输出;
    * 迭代结束，输出主要的结果费米能级以及能带信息
    * band 1: 2 个 2s
    * band 234: 4个2P电子(3*1.333)
    * 费米能级对应的是最高电子占据轨道的能量
1. `VOLUME and BASIS-vectors are now :`体系优化之后的坐标，各个方向力的大小，以及总的能量，不包含 Entropy 的能量，以及sigma 趋于 0 时的能量。
1. 计算的内存和时间等信息: `General timing and accounting informations for this job:`

# Ex7 前 6 节总结

# Ex8 氧原子能量的正确计算

![计算O原子的能量需要打开自旋极化](data/O.png)


默认情况不加自旋极化: `ISPIN = 1`
* 不计算磁矩, 所以OSZCAR中不输出`mag=`;
* band 2-4 为三个简并态, 每个能带上有 1.33 个电子，对应的 2p4;
* 此处的电子占据是不正确, 因为 2px轨道上含有两个电子，而不是把 2px上的一个电子平均分配到
3 个 p 轨道上；
* 此时并不会输出体系的磁矩

![不考虑磁性](data/O-band.png)

怎样解决原子计算中的磁矩问题

## ISPIN

因为氧原子具有磁性(电子结构中有不成对的电子), 所以要考虑自旋极化, 所以要设置ISPIN = 2 (默认为1, 不考虑自旋极化);    
自旋极化什么时候该加，什么时候用默认值呢？
* 要计算的体系具有磁性时，必须要打开自旋极化;
* 单原子的计算， O2分子（基态为三重态），以及含 Fe， Co， Ni 的体系需要考虑自旋极化。

设置自旋极化之后, OSZICAR的最后行多了一个mag字段, 对应体系的磁矩大小;  
考虑自旋极化后，电子迭代步数增加, 体系的能量降低了。  


体系的电子占据情况: `band No.`的信息会按自旋成分进行描述: 
* spin component 1, 自旋向上的$\alpha$电子;
* spin component 2, 自旋先下的$\beta$电子;

![O-spin](data/O-band-spin.png)

bnad energies是单个电子的能量, 如果计算总能量要乘以2(band上的电子数)

此时电子占据依然不合理:  
spin component 1 中含有 4 个电子，另一个中含有 2 个电子：  
* Spincomponent 1 中，能带 2-4 中 3 个 alpha 电子的能量是一样 的，这不正确，因为其中一个 2px 轨道中的 alpha 电子已经成对，能量要比 py 和 pz 低；描述失败
* spincomponent 2 中，另一个 beta 电子还是平均分配在了三个能带上面

上面的结果是由于体系的高对称性=>能量简并所造成的。 8x 8 x 8 的立方体格子，在这里具有高阶的点群对称性： O_h，
`grep 'the point symmetry' OUTCAR `;  


在 GGA 泛函中，为了获得体系更低的能量，对于原子来说，通常会采用一种 symmetry broken solution 的处理方法。需要自己手动破除;  
* 微调晶格参数;
* 默认为ISYM=2, off为0;

![](data/O-band-spin-sym.png)

### 一个电子的能量是-25, 两个电子的能量也是-25?  
* 25是单个电子的能量

* 在 alpha 电子（spin component 1）中，能带 1 是 2s 中的alpha 电子；能带 2-3 为两个简并轨道，对应 py和 pz 电子，能带 4 位px 中的 alpha 电子。这里能带 4 的能量应该比 2 和 3 要低，但结果恰恰相反，描述不合理；
* 在 beta 电子（spin component 2）中，能带 1 是 2s的 beta 电子，能带 2 为 px中的 beta 电子，此时该 beta 电子占据了一个轨道，而不是分布在三个 p 轨道上，结果是合理的;

已经是 VASP 官网对于单原子能量最为完整和准确的计算了。虽然上面 px的 alpha 电子比 py和 pz能量高，至少电子的轨道电
子占据情况是正确的，且磁矩和简并的错误已经消除。由此可见， VASP 对于单原子的电子占据情况（至少对于 O 原子来说），虽然取得了部分理想结果，但还是有不足的地方

# EX9 学会收敛点！

EDIFF: 控制电子步自洽收敛标准;  
* 在大多数的情况下， 1E-4 足以胜任，没有必要采用其他的数值，如果大伙感觉不放心，大师兄建议1E-5 即可;  

EDIFFG 控制几何优化过程的收敛情况:  
* 对于优化，我们可以使用力作为收敛标准，此时 EDIFFG 为负值。一般来说取值在-0.01 到-0.05 之间;
* 使用能量作为标准：此时， EDIFFG 为正值，一般为 0.001-0.0001

# Ex10: O2 结构搭建以及 POTCAR 注意部分 P78

CRC Handbook of Chemistry and Physics: 

Computational Chemistry Comparison and Benchmark DataBase: https://cccbdb.nist.gov/

# Ex11 VASP 中 MAGMOM 的简单设定

 O2 分子的轨道分析:  
 * 每个原子只提供一个磁矩单位

![](data/O2.png)

MAGMOM = 1*1: 第一个 1 指的是有 1 个原子；第二个 1 指的是该原子的磁矩为 1;  

* 对于简单体系来说， MAGMOM 可以采用默认值
* MAGMOM 设置的时候，初始值不要求与实验值完全一致，一般取大些（1.5 倍）比较好。
* MAGMOM 磁矩中*前后没有空格

In [23]:
%%disabled 
# POSCAR
Ni C H O
56 17 20 6

# INCCAR
ISPIN = 2
MAGMOM = 56*1 17*0 20*0 6*0

# Ex12 O2 单点计算和优化结果分析

![](data/O2-band2.png)

将band的能量按照由小到大的顺序,按由低到高的顺序填入分子轨道图: 对于$\alpha$电子的描述不准确;    
* 实验的结构不能直接用来计算其性质，只可以作为一个理想的初始值。
* 在计算一个结构的性质之前, 需要将结构进行优化

**IBRION** determines how the ions are updated and moved. 也就是说 IBRION这个参数决定了结构的优化过程  
* 一般来说，优化结构的时候有 3 个选择：
* IBRION=3：你的初始结构很差的时候；
* IBRION=2：共轭梯度算法，很可靠的一个选择，一般来说都用它。
* IBRION=1：用于小范围内稳定结构的搜索。

**如果你的体系遇到结构不收敛的时候，可以尝试下换下 IBRION 的参数。**

NSW默认为0, 表示不进行离子步优化, 自己设置为999

使用COUNTCAR查看优化后的结构参数:  
注意CONTCAR中的坐标时分数坐标, z轴相减在再乘以z轴的长度才是氧的键长

In [25]:
(0.1502935256410214-(-0.0021453774928761))*8.099999

1.234754962945667

In [28]:
kk 1

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>!kk1:active>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


# Ex15 论合理结构的重要性（三） P117

In [22]:
kk 1

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>!kk1:active>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


# add-问题

vasp笔记:
http://vasplee.lofter.com/?page=2

# add-常用命令

赝势文件的创建和查看:
* 创建赝势文件的命令: `cat ~/vasp/ikppv/POTCAR/O/POTCAR >POTCAR`
* 查看赝势文件中的元素: `grep TIT POTCAR`
* 查看赝势文件的截断能: `grep ENMAX POTCAR`
* 泛函: `grep PBE POTCAR`

批量生成10个包含相同文件的目录:  
* sigma(0.01-0.09): `for i in {2..9}; do cp -r 0.01 0.0$i; done`
* kpoints(111,666): `for i in {1..6}; do cp ../sigma/0.01 $i$i$i; done`
* poscar(888,161616): `for i in $(seq 8 2 16); do cp -r ../sigma/0.01/ $i$i$i; done`

使用不同的字符串批量替换所有文件中的字符串):  
* `for i in * ; do sed -i "3s/0.01/$i/g" $i/INCAR; done`;  
* `for i in {1..6}; do sed -i "4s/1 1 1/$i $i $i/g" $i$i$i/KPOINTS; done`
* ` for i in $(seq 8 2 16); do sed -i "3,5s/8.0/$i/g" $i$i$i/POSCAR; done`

使用cat时输出行号: `cat 0.04/INCAR -n`;   
查看自己的任务状态: `qstat`;  
查看资源的使用情况(不同集群输出的不一样): `qstat -f`

来查看当前目录下的文件夹级别信息: tree

检查任务是否算完有很多种方式: 
* `tail OUTCAR` 
* `tail results.o..`
* `watch tail OSZICAR`

In [1]:
kk 2

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$!kk2:end>$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
